In [1]:
import tensorflow as tf
print(tf.__version__)

for i in tf.config.list_physical_devices():
    print(i)

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2.1.0
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1405405255902147032,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6586313605
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6607990149170155876
 physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6586313605
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5113480717157090
 physical_device_desc: "device: 1, name: GeForce RTX 2070 SUPER, pci bus id: 0000:03:00.0, compute capability: 7.5"]

As crop_img and crop_img_to functions are missing in utils.py, we deactivated crop-related functions.

In [2]:
#unet3d/utils/sitk_utils.py
import SimpleITK as sitk
import numpy as np

def calculate_origin_offset(new_spacing, old_spacing):
    return np.subtract(new_spacing, old_spacing)/2


def sitk_resample_to_spacing(image, new_spacing=(1.0, 1.0, 1.0), interpolator=sitk.sitkLinear, default_value=0.):
    zoom_factor = np.divide(image.GetSpacing(), new_spacing)
    new_size = np.asarray(np.ceil(np.round(np.multiply(zoom_factor, image.GetSize()), decimals=5)), dtype=np.int16)
    offset = calculate_origin_offset(new_spacing, image.GetSpacing())
    reference_image = sitk_new_blank_image(size=new_size, spacing=new_spacing, direction=image.GetDirection(),
                                           origin=image.GetOrigin() + offset, default_value=default_value)
    return sitk_resample_to_image(image, reference_image, interpolator=interpolator, default_value=default_value)


def sitk_resample_to_image(image, reference_image, default_value=0., interpolator=sitk.sitkLinear, transform=None,
                           output_pixel_type=None):
    if transform is None:
        transform = sitk.Transform()
        transform.SetIdentity()
    if output_pixel_type is None:
        output_pixel_type = image.GetPixelID()
    resample_filter = sitk.ResampleImageFilter()
    resample_filter.SetInterpolator(interpolator)
    resample_filter.SetTransform(transform)
    resample_filter.SetOutputPixelType(output_pixel_type)
    resample_filter.SetDefaultPixelValue(default_value)
    resample_filter.SetReferenceImage(reference_image)
    return resample_filter.Execute(image)


def sitk_new_blank_image(size, spacing, direction, origin, default_value=0.):
    image = sitk.GetImageFromArray(np.ones(size, dtype=np.float).T * default_value)
    image.SetSpacing(spacing)
    image.SetDirection(direction)
    image.SetOrigin(origin)
    return image


def resample_to_spacing(data, spacing, target_spacing, interpolation="linear", default_value=0.):
    image = data_to_sitk_image(data, spacing=spacing)
    if interpolation is "linear":
        interpolator = sitk.sitkLinear
    elif interpolation is "nearest":
        interpolator = sitk.sitkNearestNeighbor
    else:
        raise ValueError("'interpolation' must be either 'linear' or 'nearest'. '{}' is not recognized".format(
            interpolation))
    resampled_image = sitk_resample_to_spacing(image, new_spacing=target_spacing, interpolator=interpolator,
                                               default_value=default_value)
    return sitk_image_to_data(resampled_image)


def data_to_sitk_image(data, spacing=(1., 1., 1.)):
    if len(data.shape) == 3:
        data = np.rot90(data, 1, axes=(0, 2))
    image = sitk.GetImageFromArray(data)
    image.SetSpacing(np.asarray(spacing, dtype=np.float))
    return image


def sitk_image_to_data(image):
    data = sitk.GetArrayFromImage(image)
    if len(data.shape) == 3:
        data = np.rot90(data, -1, axes=(0, 2))
    return data


In [3]:
#unet3d/utils/utils.py

import pickle
import os
import collections

import nibabel as nib
import numpy as np
from nilearn.image import reorder_img, new_img_like

def fix_shape(image):
    if image.shape[-1] == 1:
        return image.__class__(dataobj=np.squeeze(image.get_data()), affine=image.affine)
    return image


def resize(image, new_shape, interpolation="linear"):
    image = reorder_img(image, resample=interpolation)
    zoom_level = np.divide(new_shape, image.shape)
    new_spacing = np.divide(image.header.get_zooms(), zoom_level)
    new_data = resample_to_spacing(image.get_data(), image.header.get_zooms(), new_spacing,
                                   interpolation=interpolation)
    new_affine = np.copy(image.affine)
    np.fill_diagonal(new_affine, new_spacing.tolist() + [1])
    new_affine[:3, 3] += calculate_origin_offset(new_spacing, image.header.get_zooms())
    return new_img_like(image, new_data, affine=new_affine)

def read_image(in_file, image_shape=None, interpolation='linear', crop=None):
    print("Reading: {0}".format(in_file))
    image = nib.load(in_file) # J.Lee: nib.load(os.path.abspath(in_file)) -> nib.load(in_file)
    image = fix_shape(image)
#     if crop:
#         image = crop_img_to(image, crop, copy=True)
    if image_shape:
        return resize(image, new_shape=image_shape, interpolation=interpolation)
    else:
        return image


def read_image_files(image_files, image_shape=None, crop=None, label_indices=None):
    """
    :param image_files: # J.Lee: paths of image_files.
    :param image_shape: # J.Lee: (d,l,m)(155,240,240)
    # J.Lee:
    :param label_indices:
        len(image_files)-1. i.e. expected index of label(truth) image 
        (when called through: write_image_data_to_file-> reslice_image-> read_image_files)
    :param crop: 
    :param use_nearest_for_last_file: If True, will use nearest neighbor interpolation for the last file. This is used
    because the last file may be the labels file. Using linear interpolation here would mess up the labels.
    :return: 
    """
    if label_indices is None:
        label_indices = []
    elif not isinstance(label_indices, collections.Iterable) or isinstance(label_indices, str):
        label_indices = [label_indices]
    image_list = list()
    for index, image_file in enumerate(image_files):
        if (label_indices is None and (index + 1) == len(image_files)) \
                or (label_indices is not None and index in label_indices):
            interpolation = "nearest"
        else:
            interpolation = "linear"
        image_list.append(read_image(image_file, image_shape=image_shape, crop=crop, interpolation=interpolation))

    return image_list




In [4]:
#unet3d/normalize.py

import os

import numpy as np
from nilearn.image import new_img_like

# from unet3d.utils.utils import resize, read_image_files
# from .utils import crop_img, crop_img_to, read_image #!!!Missing in .utils!!!

# def get_cropping_parameters(in_files):
#     if len(in_files) > 1:
#         foreground = get_complete_foreground(in_files)
#     else:
#         foreground = get_foreground_from_set_of_files(in_files[0], return_image=True)
#     return crop_img(foreground, return_slices=True, copy=True)


def reslice_image_set(in_files, image_shape, out_files=None, label_indices=None, crop=False):
    if crop:
        pass
#         crop_slices = get_cropping_parameters([in_files])
    else:
        crop_slices = None
    images = read_image_files(in_files, image_shape=image_shape, crop=crop_slices, label_indices=label_indices)
    if out_files:
        for image, out_file in zip(images, out_files):
            image.to_filename(out_file)
        return [out_file for out_file in out_files]
        # J.Lee: original: [os.path.abspath(out_file) for out_file in out_files]
    else:
        return images

def normalize_data(data, mean, std):
    data -= mean[:, np.newaxis, np.newaxis, np.newaxis]
    data /= std[:, np.newaxis, np.newaxis, np.newaxis]
    return data
    
def normalize_data_storage(data_storage):
    means = list()
    stds = list()
    for index in range(data_storage.shape[0]):
        data = data_storage[index]
        means.append(data.mean(axis=(1, 2, 3)))
        stds.append(data.std(axis=(1, 2, 3)))
    mean = np.asarray(means).mean(axis=0)
    std = np.asarray(stds).mean(axis=0)
    for index in range(data_storage.shape[0]):
        data_storage[index] = normalize_data(data_storage[index], mean, std)
    return data_storage

In [5]:
#unet3d/data.py
import os

import numpy as np
import tables

#from .normalize import normalize_data_storage, reslice_image_set


def create_data_file(out_file, n_channels, n_samples, image_shape):
    hdf5_file = tables.open_file(out_file, mode='w')
    filters = tables.Filters(complevel=5, complib='blosc')
    data_shape = tuple([0, n_channels] + list(image_shape)) # J.Lee: (155,240,240) -> [0,4,155,240,240]
    truth_shape = tuple([0, 1] + list(image_shape)) # J.Lee: (155,240,240) -> [0,1,155,240,240]
    data_storage = hdf5_file.create_earray(hdf5_file.root, 'data', tables.Float32Atom(), shape=data_shape,
                                           filters=filters, expectedrows=n_samples)
    truth_storage = hdf5_file.create_earray(hdf5_file.root, 'truth', tables.UInt8Atom(), shape=truth_shape,
                                            filters=filters, expectedrows=n_samples)
    affine_storage = hdf5_file.create_earray(hdf5_file.root, 'affine', tables.Float32Atom(), shape=(0, 4, 4),
                                             filters=filters, expectedrows=n_samples)
    return hdf5_file, data_storage, truth_storage, affine_storage


def write_image_data_to_file(image_files, data_storage, truth_storage, image_shape, n_channels, affine_storage,
                             truth_dtype=np.uint8, crop=True):
    for set_of_files in image_files:
        # J.Lee: {set of files} includes paths of 1~4 channel images + 1truth image
        images = reslice_image_set(set_of_files, image_shape, label_indices=len(set_of_files) - 1, crop=crop)
        subject_data = [image.get_data() for image in images]
        add_data_to_storage(data_storage, truth_storage, affine_storage, subject_data, images[0].affine, n_channels,
                            truth_dtype)
    return data_storage, truth_storage


def add_data_to_storage(data_storage, truth_storage, affine_storage, subject_data, affine, n_channels, truth_dtype):
    data_storage.append(np.asarray(subject_data[:n_channels])[np.newaxis])
    truth_storage.append(np.asarray(subject_data[n_channels], dtype=truth_dtype)[np.newaxis][np.newaxis])
    affine_storage.append(np.asarray(affine)[np.newaxis])


def write_data_to_file(training_data_files, out_file, image_shape, truth_dtype=np.uint8, subject_ids=None,
                       normalize=True, crop=False): # J.Lee: crop=True -> crop=False
    """
    Takes in a set of training images and writes those images to an hdf5 file.
    :param training_data_files: List of tuples containing the training data files. The modalities should be listed in
    the same order in each tuple. The last item in each tuple must be the labeled image. 
    Example: [('sub1-T1.nii.gz', 'sub1-T2.nii.gz', 'sub1-truth.nii.gz'), 
              ('sub2-T1.nii.gz', 'sub2-T2.nii.gz', 'sub2-truth.nii.gz')]
    :param out_file: Where the hdf5 file will be written to.
    :param image_shape: Shape of the images that will be saved to the hdf5 file.
    :param truth_dtype: Default is 8-bit unsigned integer.
    :return: Location of the hdf5 file with the image data written to it. 
    """
    n_samples = len(training_data_files)
    n_channels = len(training_data_files[0]) - 1

    try:
        hdf5_file, data_storage, truth_storage, affine_storage = create_data_file(out_file,
                                                                                  n_channels=n_channels,
                                                                                  n_samples=n_samples,
                                                                                  image_shape=image_shape)
    except Exception as e:
        # If something goes wrong, delete the incomplete data file
        os.remove(out_file)
        raise e

    write_image_data_to_file(training_data_files, data_storage, truth_storage, image_shape,
                             truth_dtype=truth_dtype, n_channels=n_channels, affine_storage=affine_storage, crop=crop)
    if subject_ids:
        hdf5_file.create_array(hdf5_file.root, 'subject_ids', obj=subject_ids)
    if normalize:
        normalize_data_storage(data_storage)
    hdf5_file.close()
    return out_file


def open_data_file(filename, readwrite="r"):
    return tables.open_file(filename, readwrite)


In [6]:
#modified from N.Freidman's preprocessing code
data_dir = 'X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/'
DATA_HGG = data_dir+'HGG/'
DATA_LGG = data_dir+'LGG/'

def modality_sorter(file_path, modality_order = ['flair', 't1','t1ce', 't2', 'seg']):
    '''used as key to sort {file path} by {modality_order}'''
    for i, modality in enumerate(modality_order):
        if modality in file_path: return i
        
hgg_paths = []
for dirpath, dirnames, files in os.walk(DATA_HGG):
    if ('BraTS19' in dirpath):
        file_paths = [dirpath+'/'+file for file in files if ('nii.gz' in file) and ('nb4' not in file)] # J.Lee: include N4Bias images if you want
        file_paths.sort(key=modality_sorter)
        hgg_paths.append(tuple(file_paths))

lgg_paths = []
for dirpath, dirnames, files in os.walk(DATA_LGG):
    if ('BraTS19' in dirpath):
        file_paths = [dirpath+'/'+file for file in files if ('nii.gz' in file) and ('nb4' not in file)] # J.Lee: include N4Bias images if you want
        file_paths.sort(key=modality_sorter)
        lgg_paths.append(tuple(file_paths))

training_data_files = hgg_paths+lgg_paths

import pandas as pd
from IPython.display import display
pd.set_option('max_colwidth', 120)

display(pd.DataFrame(training_data_files[1]))
display(pd.DataFrame(training_data_files[-1]))

out_file = data_dir+'htf5/'
image_shape = (155,240,240)

,0
0,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_flair.nii.gz
1,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t1.nii.gz
2,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t1ce.nii.gz
3,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t2.nii.gz
4,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_seg.nii.gz


,0
0,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS19_TMC_09043_1_flair.nii.gz
1,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS19_TMC_09043_1_t1.nii.gz
2,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS19_TMC_09043_1_t1ce.nii.gz
3,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS19_TMC_09043_1_t2.nii.gz
4,X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS19_TMC_09043_1_seg.nii.gz


In [7]:
# def write_data_to_file(training_data_files, out_file, image_shape, truth_dtype=np.uint8, subject_ids=None,
#                        normalize=True, crop=False)

training_data_files = training_data_files
outfiles = 'X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/h5_outfiles.h5'
image_shape = (155,240,240)

#write_data_to_file(training_data_files, outfiles, image_shape)

c:\iamedic\myenv_tf21\lib\site-packages\tables\leaf.py:410: PerformanceWarning: The Leaf ``/data`` is exceeding the maximum recommended rowsize (104857600 bytes);
be ready to see PyTables asking for *lots* of memory and possibly slow
I/O.  You may want to reduce the rowsize by trimming the value of
dimensions that are orthogonal (and preferably close) to the *main*
dimension of this leave.  Alternatively, in case you have specified a
very small/large chunksize, you may want to increase/decrease it.
  PerformanceWarning)
c:\iamedic\myenv_tf21\lib\site-packages\ipykernel_launcher.py:55: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
c:\iamedic\myenv_tf21\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(

Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_10_1/BraTS19_2013_10_1_seg.nii.gz


c:\iamedic\myenv_tf21\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0


Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_11_1/BraTS19_2013_11_1_seg.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_12_1/BraTS19_2013_12_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_12_1/BraTS19_2013_12_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_12_1/BraTS19_2013_12_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_12_1/BraTS19_2013_12_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/M

Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_3_1/BraTS19_2013_3_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_3_1/BraTS19_2013_3_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_3_1/BraTS19_2013_3_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_3_1/BraTS19_2013_3_1_seg.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_4_1/BraTS19_2013_4_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_4_1/BraTS19_2013_4_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_4_1/BraTS19_2013_4_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_4_1/BraTS19_2013_4_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_2013_4_1/BraTS19_2013_4_1_seg.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Dat

...

Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_103_1/BraTS19_TCIA10_103_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_103_1/BraTS19_TCIA10_103_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_103_1/BraTS19_TCIA10_103_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_103_1/BraTS19_TCIA10_103_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_103_1/BraTS19_TCIA10_103_1_seg.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_109_1/BraTS19_TCIA10_109_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_109_1/BraTS19_TCIA10_109_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_109_1/BraTS19_TCIA10_109_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_109_1/Bra

Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_325_1/BraTS19_TCIA10_325_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_325_1/BraTS19_TCIA10_325_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_325_1/BraTS19_TCIA10_325_1_seg.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_330_1/BraTS19_TCIA10_330_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_330_1/BraTS19_TCIA10_330_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_330_1/BraTS19_TCIA10_330_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_330_1/BraTS19_TCIA10_330_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_330_1/BraTS19_TCIA10_330_1_seg.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_346_1/BraTS

Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_628_1/BraTS19_TCIA10_628_1_seg.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_629_1/BraTS19_TCIA10_629_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_629_1/BraTS19_TCIA10_629_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_629_1/BraTS19_TCIA10_629_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_629_1/BraTS19_TCIA10_629_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_629_1/BraTS19_TCIA10_629_1_seg.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_632_1/BraTS19_TCIA10_632_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_632_1/BraTS19_TCIA10_632_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA10_632_1/BraT

Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_621_1/BraTS19_TCIA13_621_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_621_1/BraTS19_TCIA13_621_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_621_1/BraTS19_TCIA13_621_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_621_1/BraTS19_TCIA13_621_1_seg.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_623_1/BraTS19_TCIA13_623_1_flair.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_623_1/BraTS19_TCIA13_623_1_t1.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_623_1/BraTS19_TCIA13_623_1_t1ce.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_623_1/BraTS19_TCIA13_623_1_t2.nii.gz
Reading: X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_623_1/BraTS1

'X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/h5_outfiles.h5'

In [23]:
#generator related functions

import tensorflow

#utils/utils.py

def pickle_dump(item, out_file):
    with open(out_file, "wb") as opened_file:
        pickle.dump(item, opened_file)


def pickle_load(in_file):
    with open(in_file, "rb") as opened_file:
        return pickle.load(opened_file)

#utils/patches.py
# import numpy as np


def compute_patch_indices(image_shape, patch_size, overlap, start=None):
    if isinstance(overlap, int):
        overlap = np.asarray([overlap] * len(image_shape))
    if start is None:
        n_patches = np.ceil(image_shape / (patch_size - overlap))
        overflow = (patch_size - overlap) * n_patches - image_shape + overlap
        start = -np.ceil(overflow/2)
    elif isinstance(start, int):
        start = np.asarray([start] * len(image_shape))
    stop = image_shape + start
    step = patch_size - overlap
    return get_set_of_patch_indices(start, stop, step)


def get_set_of_patch_indices(start, stop, step):
    return np.asarray(np.mgrid[start[0]:stop[0]:step[0], start[1]:stop[1]:step[1],
                               start[2]:stop[2]:step[2]].reshape(3, -1).T, dtype=np.int)


def get_random_patch_index(image_shape, patch_shape):
    """
    Returns a random corner index for a patch. If this is used during training, the middle pixels will be seen by
    the model way more often than the edge pixels (which is probably a bad thing).
    :param image_shape: Shape of the image
    :param patch_shape: Shape of the patch
    :return: a tuple containing the corner index which can be used to get a patch from an image
    """
    return get_random_nd_index(np.subtract(image_shape, patch_shape))


def get_random_nd_index(index_max):
    return tuple([np.random.choice(index_max[index] + 1) for index in range(len(index_max))])


def get_patch_from_3d_data(data, patch_shape, patch_index):
    """
    Returns a patch from a numpy array.
    :param data: numpy array from which to get the patch.
    :param patch_shape: shape/size of the patch.
    :param patch_index: corner index of the patch.
    :return: numpy array take from the data with the patch shape specified.
    """
    patch_index = np.asarray(patch_index, dtype=np.int16)
    patch_shape = np.asarray(patch_shape)
    image_shape = data.shape[-3:]
    if np.any(patch_index < 0) or np.any((patch_index + patch_shape) > image_shape):
        data, patch_index = fix_out_of_bound_patch_attempt(data, patch_shape, patch_index)
    return data[..., patch_index[0]:patch_index[0]+patch_shape[0], patch_index[1]:patch_index[1]+patch_shape[1],
                patch_index[2]:patch_index[2]+patch_shape[2]]


def fix_out_of_bound_patch_attempt(data, patch_shape, patch_index, ndim=3):
    """
    Pads the data and alters the patch index so that a patch will be correct.
    :param data:
    :param patch_shape:
    :param patch_index:
    :return: padded data, fixed patch index
    """
    image_shape = data.shape[-ndim:]
    pad_before = np.abs((patch_index < 0) * patch_index)
    pad_after = np.abs(((patch_index + patch_shape) > image_shape) * ((patch_index + patch_shape) - image_shape))
    pad_args = np.stack([pad_before, pad_after], axis=1)
    if pad_args.shape[0] < len(data.shape):
        pad_args = [[0, 0]] * (len(data.shape) - pad_args.shape[0]) + pad_args.tolist()
    data = np.pad(data, pad_args, mode="edge")
    patch_index += pad_before
    return data, patch_index


def reconstruct_from_patches(patches, patch_indices, data_shape, default_value=0):
    """
    Reconstructs an array of the original shape from the lists of patches and corresponding patch indices. Overlapping
    patches are averaged.
    :param patches: List of numpy array patches.
    :param patch_indices: List of indices that corresponds to the list of patches.
    :param data_shape: Shape of the array from which the patches were extracted.
    :param default_value: The default value of the resulting data. if the patch coverage is complete, this value will
    be overwritten.
    :return: numpy array containing the data reconstructed by the patches.
    """
    data = np.ones(data_shape) * default_value
    image_shape = data_shape[-3:]
    count = np.zeros(data_shape, dtype=np.int)
    for patch, index in zip(patches, patch_indices):
        image_patch_shape = patch.shape[-3:]
        if np.any(index < 0):
            fix_patch = np.asarray((index < 0) * np.abs(index), dtype=np.int)
            patch = patch[..., fix_patch[0]:, fix_patch[1]:, fix_patch[2]:]
            index[index < 0] = 0
        if np.any((index + image_patch_shape) >= image_shape):
            fix_patch = np.asarray(image_patch_shape - (((index + image_patch_shape) >= image_shape)
                                                        * ((index + image_patch_shape) - image_shape)), dtype=np.int)
            patch = patch[..., :fix_patch[0], :fix_patch[1], :fix_patch[2]]
        patch_index = np.zeros(data_shape, dtype=np.bool)
        patch_index[...,
                    index[0]:index[0]+patch.shape[-3],
                    index[1]:index[1]+patch.shape[-2],
                    index[2]:index[2]+patch.shape[-1]] = True
        patch_data = np.zeros(data_shape)
        patch_data[patch_index] = patch.flatten()

        new_data_index = np.logical_and(patch_index, np.logical_not(count > 0))
        data[new_data_index] = patch_data[new_data_index]

        averaged_data_index = np.logical_and(patch_index, count > 0)
        if np.any(averaged_data_index):
            data[averaged_data_index] = (data[averaged_data_index] * count[averaged_data_index] + patch_data[averaged_data_index]) / (count[averaged_data_index] + 1)
        count[patch_index] += 1
    return data

#augment.py
import numpy as np
import nibabel as nib
from nilearn.image import new_img_like, resample_to_img
import random
import itertools


def scale_image(image, scale_factor):
    scale_factor = np.asarray(scale_factor)
    new_affine = np.copy(image.affine)
    new_affine[:3, :3] = image.affine[:3, :3] * scale_factor
    new_affine[:, 3][:3] = image.affine[:, 3][:3] + (image.shape * np.diag(image.affine)[:3] * (1 - scale_factor)) / 2
    return new_img_like(image, data=image.get_data(), affine=new_affine)


def flip_image(image, axis):
    try:
        new_data = np.copy(image.get_data())
        for axis_index in axis:
            new_data = np.flip(new_data, axis=axis_index)
    except TypeError:
        new_data = np.flip(image.get_data(), axis=axis)
    return new_img_like(image, data=new_data)


def random_flip_dimensions(n_dimensions):
    axis = list()
    for dim in range(n_dimensions):
        if random_boolean():
            axis.append(dim)
    return axis


def random_scale_factor(n_dim=3, mean=1, std=0.25):
    return np.random.normal(mean, std, n_dim)


def random_boolean():
    return np.random.choice([True, False])


def distort_image(image, flip_axis=None, scale_factor=None):
    if flip_axis:
        image = flip_image(image, flip_axis)
    if scale_factor is not None:
        image = scale_image(image, scale_factor)
    return image


def augment_data(data, truth, affine, scale_deviation=None, flip=True):
    n_dim = len(truth.shape)
    if scale_deviation:
        scale_factor = random_scale_factor(n_dim, std=scale_deviation)
    else:
        scale_factor = None
    if flip:
        flip_axis = random_flip_dimensions(n_dim)
    else:
        flip_axis = None
    data_list = list()
    for data_index in range(data.shape[0]):
        image = get_image(data[data_index], affine)
        data_list.append(resample_to_img(distort_image(image, flip_axis=flip_axis,
                                                       scale_factor=scale_factor), image,
                                         interpolation="continuous").get_data())
    data = np.asarray(data_list)
    truth_image = get_image(truth, affine)
    truth_data = resample_to_img(distort_image(truth_image, flip_axis=flip_axis, scale_factor=scale_factor),
                                 truth_image, interpolation="nearest").get_data()
    return data, truth_data


def get_image(data, affine, nib_class=nib.Nifti1Image):
    return nib_class(dataobj=data, affine=affine)


def generate_permutation_keys():
    """
    This function returns a set of "keys" that represent the 48 unique rotations &
    reflections of a 3D matrix.

    Each item of the set is a tuple:
    ((rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose)

    As an example, ((0, 1), 0, 1, 0, 1) represents a permutation in which the data is
    rotated 90 degrees around the z-axis, then reversed on the y-axis, and then
    transposed.

    48 unique rotations & reflections:
    https://en.wikipedia.org/wiki/Octahedral_symmetry#The_isometries_of_the_cube
    """
    return set(itertools.product(
        itertools.combinations_with_replacement(range(2), 2), range(2), range(2), range(2), range(2)))


def random_permutation_key():
    """
    Generates and randomly selects a permutation key. See the documentation for the
    "generate_permutation_keys" function.
    """
    return random.choice(list(generate_permutation_keys()))


def permute_data(data, key):
    """
    Permutes the given data according to the specification of the given key. Input data
    must be of shape (n_modalities, x, y, z).

    Input key is a tuple: (rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose)

    As an example, ((0, 1), 0, 1, 0, 1) represents a permutation in which the data is
    rotated 90 degrees around the z-axis, then reversed on the y-axis, and then
    transposed.
    """
    data = np.copy(data)
    (rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose = key

    if rotate_y != 0:
        data = np.rot90(data, rotate_y, axes=(1, 3))
    if rotate_z != 0:
        data = np.rot90(data, rotate_z, axes=(2, 3))
    if flip_x:
        data = data[:, ::-1]
    if flip_y:
        data = data[:, :, ::-1]
    if flip_z:
        data = data[:, :, :, ::-1]
    if transpose:
        for i in range(data.shape[0]):
            data[i] = data[i].T
    return data


def random_permutation_x_y(x_data, y_data):
    """
    Performs random permutation on the data.
    :param x_data: numpy array containing the data. Data must be of shape (n_modalities, x, y, z).
    :param y_data: numpy array containing the data. Data must be of shape (n_modalities, x, y, z).
    :return: the permuted data
    """
    key = random_permutation_key()
    return permute_data(x_data, key), permute_data(y_data, key)


def reverse_permute_data(data, key):
    key = reverse_permutation_key(key)
    data = np.copy(data)
    (rotate_y, rotate_z), flip_x, flip_y, flip_z, transpose = key

    if transpose:
        for i in range(data.shape[0]):
            data[i] = data[i].T
    if flip_z:
        data = data[:, :, :, ::-1]
    if flip_y:
        data = data[:, :, ::-1]
    if flip_x:
        data = data[:, ::-1]
    if rotate_z != 0:
        data = np.rot90(data, rotate_z, axes=(2, 3))
    if rotate_y != 0:
        data = np.rot90(data, rotate_y, axes=(1, 3))
    return data


def reverse_permutation_key(key):
    rotation = tuple([-rotate for rotate in key[0]])
    return rotation, key[1], key[2], key[3], key[4]



#generator.py
import os
import copy
from random import shuffle
import itertools

import numpy as np

# from .utils import pickle_dump, pickle_load
# from .utils.patches import compute_patch_indices, get_random_nd_index, get_patch_from_3d_data
# from .augment import augment_data, random_permutation_x_y


def get_training_and_validation_generators(data_file, batch_size, n_labels, training_keys_file, validation_keys_file,
                                           data_split=0.8, overwrite=False, labels=None, augment=False,
                                           augment_flip=True, augment_distortion_factor=0.25, patch_shape=None,
                                           validation_patch_overlap=0, training_patch_start_offset=None,
                                           validation_batch_size=None, skip_blank=True, permute=False):
    """
    Creates the training and validation generators that can be used when training the model.
    :param skip_blank: If True, any blank (all-zero) label images/patches will be skipped by the data generator.
    :param validation_batch_size: Batch size for the validation data.
    :param training_patch_start_offset: Tuple of length 3 containing integer values. Training data will randomly be
    offset by a number of pixels between (0, 0, 0) and the given tuple. (default is None)
    :param validation_patch_overlap: Number of pixels/voxels that will be overlapped in the validation data. (requires
    patch_shape to not be None)
    :param patch_shape: Shape of the data to return with the generator. If None, the whole image will be returned.
    (default is None)
    :param augment_flip: if True and augment is True, then the data will be randomly flipped along the x, y and z axis
    :param augment_distortion_factor: if augment is True, this determines the standard deviation from the original
    that the data will be distorted (in a stretching or shrinking fashion). Set to None, False, or 0 to prevent the
    augmentation from distorting the data in this way.
    :param augment: If True, training data will be distorted on the fly so as to avoid over-fitting.
    :param labels: List or tuple containing the ordered label values in the image files. The length of the list or tuple
    should be equal to the n_labels value.
    Example: (10, 25, 50)
    The data generator would then return binary truth arrays representing the labels 10, 25, and 30 in that order.
    :param data_file: hdf5 file to load the data from.
    :param batch_size: Size of the batches that the training generator will provide.
    :param n_labels: Number of binary labels.
    :param training_keys_file: Pickle file where the index locations of the training data will be stored.
    :param validation_keys_file: Pickle file where the index locations of the validation data will be stored.
    :param data_split: How the training and validation data will be split. 0 means all the data will be used for
    validation and none of it will be used for training. 1 means that all the data will be used for training and none
    will be used for validation. Default is 0.8 or 80%.
    :param overwrite: If set to True, previous files will be overwritten. The default mode is false, so that the
    training and validation splits won't be overwritten when rerunning model training.
    :param permute: will randomly permute the data (data must be 3D cube)
    :return: Training data generator, validation data generator, number of training steps, number of validation steps
    """
    if not validation_batch_size:
        validation_batch_size = batch_size

    training_list, validation_list = get_validation_split(data_file,
                                                          data_split=data_split,
                                                          overwrite=overwrite,
                                                          training_file=training_keys_file,
                                                          validation_file=validation_keys_file)

    training_generator = data_generator(data_file, training_list,
                                        batch_size=batch_size,
                                        n_labels=n_labels,
                                        labels=labels,
                                        augment=augment,
                                        augment_flip=augment_flip,
                                        augment_distortion_factor=augment_distortion_factor,
                                        patch_shape=patch_shape,
                                        patch_overlap=0,
                                        patch_start_offset=training_patch_start_offset,
                                        skip_blank=skip_blank,
                                        permute=permute)
    validation_generator = data_generator(data_file, validation_list,
                                          batch_size=validation_batch_size,
                                          n_labels=n_labels,
                                          labels=labels,
                                          patch_shape=patch_shape,
                                          patch_overlap=validation_patch_overlap,
                                          skip_blank=skip_blank)

    # Set the number of training and testing samples per epoch correctly
    num_training_steps = get_number_of_steps(get_number_of_patches(data_file, training_list, patch_shape,
                                                                   skip_blank=skip_blank,
                                                                   patch_start_offset=training_patch_start_offset,
                                                                   patch_overlap=0), batch_size)
    print("Number of training steps: ", num_training_steps)

    num_validation_steps = get_number_of_steps(get_number_of_patches(data_file, validation_list, patch_shape,
                                                                     skip_blank=skip_blank,
                                                                     patch_overlap=validation_patch_overlap),
                                               validation_batch_size)
    print("Number of validation steps: ", num_validation_steps)

    return training_generator, validation_generator, num_training_steps, num_validation_steps


def get_number_of_steps(n_samples, batch_size):
    if n_samples <= batch_size:
        return n_samples
    elif np.remainder(n_samples, batch_size) == 0:
        return n_samples//batch_size
    else:
        return n_samples//batch_size + 1


def get_validation_split(data_file, training_file, validation_file, data_split=0.8, overwrite=False):
    """
    Splits the data into the training and validation indices list.
    :param data_file: pytables hdf5 data file
    :param training_file:
    :param validation_file:
    :param data_split:
    :param overwrite:
    :return:
    """
    if overwrite or not os.path.exists(training_file):
        print("Creating validation split...")
        nb_samples = data_file.root.data.shape[0]
        sample_list = list(range(nb_samples))
        training_list, validation_list = split_list(sample_list, split=data_split)
        pickle_dump(training_list, training_file)
        pickle_dump(validation_list, validation_file)
        return training_list, validation_list
    else:
        print("Loading previous validation split...")
        return pickle_load(training_file), pickle_load(validation_file)


def split_list(input_list, split=0.8, shuffle_list=True):
    if shuffle_list:
        shuffle(input_list)
    n_training = int(len(input_list) * split)
    training = input_list[:n_training]
    testing = input_list[n_training:]
    return training, testing


def data_generator(data_file, index_list, batch_size=1, n_labels=1, labels=None, augment=False, augment_flip=True,
                   augment_distortion_factor=0.25, patch_shape=None, patch_overlap=0, patch_start_offset=None,
                   shuffle_index_list=True, skip_blank=True, permute=False):
    orig_index_list = index_list
    while True:
        x_list = list()
        y_list = list()
        if patch_shape:
            index_list = create_patch_index_list(orig_index_list, data_file.root.data.shape[-3:], patch_shape,
                                                 patch_overlap, patch_start_offset)
        else:
            index_list = copy.copy(orig_index_list)

        if shuffle_index_list:
            shuffle(index_list)
        while len(index_list) > 0:
            index = index_list.pop()
            add_data(x_list, y_list, data_file, index, augment=augment, augment_flip=augment_flip,
                     augment_distortion_factor=augment_distortion_factor, patch_shape=patch_shape,
                     skip_blank=skip_blank, permute=permute)
            if len(x_list) == batch_size or (len(index_list) == 0 and len(x_list) > 0):
                yield convert_data(x_list, y_list, n_labels=n_labels, labels=labels)
                x_list = list()
                y_list = list()


def get_number_of_patches(data_file, index_list, patch_shape=None, patch_overlap=0, patch_start_offset=None,
                          skip_blank=True):
    if patch_shape:
        index_list = create_patch_index_list(index_list, data_file.root.data.shape[-3:], patch_shape, patch_overlap,
                                             patch_start_offset)
        count = 0
        for index in index_list:
            x_list = list()
            y_list = list()
            add_data(x_list, y_list, data_file, index, skip_blank=skip_blank, patch_shape=patch_shape)
            if len(x_list) > 0:
                count += 1
        return count
    else:
        return len(index_list)


def create_patch_index_list(index_list, image_shape, patch_shape, patch_overlap, patch_start_offset=None):
    patch_index = list()
    for index in index_list:
        if patch_start_offset is not None:
            random_start_offset = np.negative(get_random_nd_index(patch_start_offset))
            patches = compute_patch_indices(image_shape, patch_shape, overlap=patch_overlap, start=random_start_offset)
        else:
            patches = compute_patch_indices(image_shape, patch_shape, overlap=patch_overlap)
        patch_index.extend(itertools.product([index], patches))
    return patch_index


def add_data(x_list, y_list, data_file, index, augment=False, augment_flip=False, augment_distortion_factor=0.25,
             patch_shape=False, skip_blank=True, permute=False):
    """
    Adds data from the data file to the given lists of feature and target data
    :param skip_blank: Data will not be added if the truth vector is all zeros (default is True).
    :param patch_shape: Shape of the patch to add to the data lists. If None, the whole image will be added.
    :param x_list: list of data to which data from the data_file will be appended.
    :param y_list: list of data to which the target data from the data_file will be appended.
    :param data_file: hdf5 data file.
    :param index: index of the data file from which to extract the data.
    :param augment: if True, data will be augmented according to the other augmentation parameters (augment_flip and
    augment_distortion_factor)
    :param augment_flip: if True and augment is True, then the data will be randomly flipped along the x, y and z axis
    :param augment_distortion_factor: if augment is True, this determines the standard deviation from the original
    that the data will be distorted (in a stretching or shrinking fashion). Set to None, False, or 0 to prevent the
    augmentation from distorting the data in this way.
    :param permute: will randomly permute the data (data must be 3D cube)
    :return:
    """
    data, truth = get_data_from_file(data_file, index, patch_shape=patch_shape)
    if augment:
        if patch_shape is not None:
            affine = data_file.root.affine[index[0]]
        else:
            affine = data_file.root.affine[index]
        data, truth = augment_data(data, truth, affine, flip=augment_flip, scale_deviation=augment_distortion_factor)

    if permute:
        if data.shape[-3] != data.shape[-2] or data.shape[-2] != data.shape[-1]:
            raise ValueError("To utilize permutations, data array must be in 3D cube shape with all dimensions having "
                             "the same length.")
        data, truth = random_permutation_x_y(data, truth[np.newaxis])
    else:
        truth = truth[np.newaxis]

    if not skip_blank or np.any(truth != 0):
        x_list.append(data)
        y_list.append(truth)


def get_data_from_file(data_file, index, patch_shape=None):
    if patch_shape:
        index, patch_index = index
        data, truth = get_data_from_file(data_file, index, patch_shape=None)
        x = get_patch_from_3d_data(data, patch_shape, patch_index)
        y = get_patch_from_3d_data(truth, patch_shape, patch_index)
    else:
        x, y = data_file.root.data[index], data_file.root.truth[index, 0]
    return x, y


def convert_data(x_list, y_list, n_labels=1, labels=None):
    x = np.asarray(x_list)
    y = np.asarray(y_list)
    if n_labels == 1:
        y[y > 0] = 1
    elif n_labels > 1:
        y = get_multi_class_labels(y, n_labels=n_labels, labels=labels)
    return x, y


def get_multi_class_labels(data, n_labels, labels=None):
    """
    Translates a label map into a set of binary labels.
    :param data: numpy array containing the label map with shape: (n_samples, 1, ...).
    :param n_labels: number of labels.
    :param labels: integer values of the labels.
    :return: binary numpy array of shape: (n_samples, n_labels, ...)
    """
    new_shape = [data.shape[0], n_labels] + list(data.shape[2:])
    y = np.zeros(new_shape, np.int8)
    for label_index in range(n_labels):
        if labels is not None:
            y[:, label_index][data[:, 0] == labels[label_index]] = 1
        else:
            y[:, label_index][data[:, 0] == (label_index + 1)] = 1
    return y


In [24]:
#metrics.py
from functools import partial

from tensorflow.keras import backend as K


def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = tf.cast(K.flatten(y_true), tf.float32, name=None)# K.flatten(y_true)
    y_pred_f = tf.cast(K.flatten(y_pred), tf.float32, name=None)
    #J.Lee: original: y_pred_f = K.flatten(y_pred)
    #to prevent Value error. original y_pred is int8 and cannot be multiplied with fp32(y_true)
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)


def weighted_dice_coefficient(y_true, y_pred, axis=(-3, -2, -1), smooth=0.00001):
    """
    Weighted dice coefficient. Default axis assumes a "channels first" data structure
    :param smooth:
    :param y_true:
    :param y_pred:
    :param axis:
    :return:
    """
    return K.mean(2. * (K.sum(y_true * y_pred,
                              axis=axis) + smooth/2)/(K.sum(y_true,
                                                            axis=axis) + K.sum(y_pred,
                                                                               axis=axis) + smooth))


def weighted_dice_coefficient_loss(y_true, y_pred):
    return -weighted_dice_coefficient(y_true, y_pred)


def label_wise_dice_coefficient(y_true, y_pred, label_index):
    return dice_coefficient(y_true[:, label_index], y_pred[:, label_index])


def get_label_dice_coefficient_function(label_index):
    f = partial(label_wise_dice_coefficient, label_index=label_index)
    f.__setattr__('__name__', 'label_{0}_dice_coef'.format(label_index))
    return f


dice_coef = dice_coefficient
dice_coef_loss = dice_coefficient_loss


#training.py

import math
from functools import partial

from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model

# from unet3d.metrics import (dice_coefficient, dice_coefficient_loss, dice_coef, dice_coef_loss,
#                             weighted_dice_coefficient_loss, weighted_dice_coefficient)

K.set_image_data_format('channels_first')
# J.Lee: K.set_image_dim_ordering('th') -> K.set_image_data_format('channels_first') for use in keras ver2.x


# learning rate schedule
def step_decay(epoch, initial_lrate, drop, epochs_drop):
    return initial_lrate * math.pow(drop, math.floor((1+epoch)/float(epochs_drop)))


def get_callbacks(model_file, initial_learning_rate=0.0001, learning_rate_drop=0.5, learning_rate_epochs=None,
                  learning_rate_patience=50, logging_file="training.log", verbosity=1,
                  early_stopping_patience=None):
    callbacks = list()
    callbacks.append(ModelCheckpoint(model_file, save_best_only=True))
    callbacks.append(CSVLogger(logging_file, append=True))
    if learning_rate_epochs:
        callbacks.append(LearningRateScheduler(partial(step_decay, initial_lrate=initial_learning_rate,
                                                       drop=learning_rate_drop, epochs_drop=learning_rate_epochs)))
    else:
        callbacks.append(ReduceLROnPlateau(factor=learning_rate_drop, patience=learning_rate_patience,
                                           verbose=verbosity))
    if early_stopping_patience:
        callbacks.append(EarlyStopping(verbose=verbosity, patience=early_stopping_patience))
    return callbacks


def load_old_model(model_file):
    print("Loading pre-trained model")
    custom_objects = {'dice_coefficient_loss': dice_coefficient_loss, 'dice_coefficient': dice_coefficient,
                      'dice_coef': dice_coef, 'dice_coef_loss': dice_coef_loss,
                      'weighted_dice_coefficient': weighted_dice_coefficient,
                      'weighted_dice_coefficient_loss': weighted_dice_coefficient_loss}
    try:
        from keras_contrib.layers import InstanceNormalization
        custom_objects["InstanceNormalization"] = InstanceNormalization
    except ImportError:
        pass
    try:
        return load_model(model_file, custom_objects=custom_objects)
    except ValueError as error:
        if 'InstanceNormalization' in str(error):
            raise ValueError(str(error) + "\n\nPlease install keras-contrib to use InstanceNormalization:\n"
                                          "'pip install git+https://www.github.com/keras-team/keras-contrib.git'")
        else:
            raise error


def train_model(model, model_file, training_generator, validation_generator, steps_per_epoch, validation_steps,
                initial_learning_rate=0.001, learning_rate_drop=0.5, learning_rate_epochs=None, n_epochs=500,
                learning_rate_patience=20, early_stopping_patience=None):
    """
    Train a Keras model.
    :param early_stopping_patience: If set, training will end early if the validation loss does not improve after the
    specified number of epochs.
    :param learning_rate_patience: If learning_rate_epochs is not set, the learning rate will decrease if the validation
    loss does not improve after the specified number of epochs. (default is 20)
    :param model: Keras model that will be trained.
    :param model_file: Where to save the Keras model.
    :param training_generator: Generator that iterates through the training data.
    :param validation_generator: Generator that iterates through the validation data.
    :param steps_per_epoch: Number of batches that the training generator will provide during a given epoch.
    :param validation_steps: Number of batches that the validation generator will provide during a given epoch.
    :param initial_learning_rate: Learning rate at the beginning of training.
    :param learning_rate_drop: How much at which to the learning rate will decay.
    :param learning_rate_epochs: Number of epochs after which the learning rate will drop.
    :param n_epochs: Total number of epochs to train the model.
    :return: 
    """
    model.fit(training_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=n_epochs,
                    validation_data=validation_generator,
                    validation_steps=validation_steps,
                    callbacks=get_callbacks(model_file,
                                            initial_learning_rate=initial_learning_rate,
                                            learning_rate_drop=learning_rate_drop,
                                            learning_rate_epochs=learning_rate_epochs,
                                            learning_rate_patience=learning_rate_patience,
                                            early_stopping_patience=early_stopping_patience))
# J.Lee:original:
#     model.fit_generator(generator=training_generator,
#                         steps_per_epoch=steps_per_epoch,
#                         epochs=n_epochs,
#                         validation_data=validation_generator,
#                         validation_steps=validation_steps,
#                         callbacks=get_callbacks(model_file,
#                                                 initial_learning_rate=initial_learning_rate,
#                                                 learning_rate_drop=learning_rate_drop,
#                                                 learning_rate_epochs=learning_rate_epochs,
#                                                 learning_rate_patience=learning_rate_patience,
#                                                 early_stopping_patience=early_stopping_patience))


In [25]:
#model/unet.py

import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras import Input, Model #J.Lee:original_code: from keras.engine import Input, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose
#J.Lee:original_code: Deconvolution3D instead of Conv3DTranspose
from tensorflow.keras.optimizers import Adam

#import tensorflow_addons as tfa #J.Lee: extracode added for InstanceNormalization 

# from unet3d.metrics import dice_coefficient_loss, get_label_dice_coefficient_function, dice_coefficient

K.set_image_data_format("channels_first")

try:
    from keras.engine import merge
except ImportError:
    from tensorflow.keras.layers import concatenate 
    #J.Lee:original_code: from keras.layers.merge import concatenate 

def unet_model_3d(input_shape, pool_size=(2, 2, 2), n_labels=1, initial_learning_rate=0.00001, deconvolution=False,
                  depth=4, n_base_filters=32, include_label_wise_dice_coefficients=False, metrics=dice_coefficient,
                  batch_normalization=False, activation_name="sigmoid"):
    """
    Builds the 3D UNet Keras model.f
    :param metrics: List metrics to be calculated during model training (default is dice coefficient).
    :param include_label_wise_dice_coefficients: If True and n_labels is greater than 1, model will report the dice
    coefficient for each label as metric.
    :param n_base_filters: The number of filters that the first layer in the convolution network will have. Following
    layers will contain a multiple of this number. Lowering this number will likely reduce the amount of memory required
    to train the model.
    :param depth: indicates the depth of the U-shape for the model. The greater the depth, the more max pooling
    layers will be added to the model. Lowering the depth may reduce the amount of memory required for training.
    :param input_shape: Shape of the input data (n_chanels, x_size, y_size, z_size). The x, y, and z sizes must be
    divisible by the pool size to the power of the depth of the UNet, that is pool_size^depth.
    :param pool_size: Pool size for the max pooling operations.
    :param n_labels: Number of binary labels that the model is learning.
    :param initial_learning_rate: Initial learning rate for the model. This will be decayed during training.
    :param deconvolution: If set to True, will use transpose convolution(deconvolution) instead of up-sampling. This
    increases the amount memory required during training.
    :return: Untrained 3D UNet Model
    """
    inputs = Input(input_shape)
    current_layer = inputs
    levels = list()

    # add levels with max pooling
    for layer_depth in range(depth):
        layer1 = create_convolution_block(input_layer=current_layer, n_filters=n_base_filters*(2**layer_depth),
                                          batch_normalization=batch_normalization)
        layer2 = create_convolution_block(input_layer=layer1, n_filters=n_base_filters*(2**layer_depth)*2,
                                          batch_normalization=batch_normalization)
        if layer_depth < depth - 1:
            current_layer = MaxPooling3D(pool_size=pool_size)(layer2)
            levels.append([layer1, layer2, current_layer])
        else:
            current_layer = layer2
            levels.append([layer1, layer2])

    # add levels with up-convolution or up-sampling
    for layer_depth in range(depth-2, -1, -1):
        up_convolution = get_up_convolution(pool_size=pool_size, deconvolution=deconvolution,
                                            #J.Lee:original: n_filters=current_layer._keras_shape[1]
                                            n_filters= tensorflow.keras.backend.int_shape(current_layer)[1])(current_layer)
        concat = concatenate([up_convolution, levels[layer_depth][1]], axis=1)
        current_layer = create_convolution_block(n_filters= tensorflow.keras.backend.int_shape(levels[layer_depth][1])[1],
                                                 # J.Lee:original: levels[layer_depth][1]._keras_shape[1],
                                                 input_layer=concat, batch_normalization=batch_normalization)
        current_layer = create_convolution_block(n_filters= tensorflow.keras.backend.int_shape(levels[layer_depth][1])[1],
                                                 # J.Lee:original: levels[layer_depth][1]._keras_shape[1],
                                                 input_layer=current_layer,
                                                 batch_normalization=batch_normalization)

    final_convolution = Conv3D(n_labels, (1, 1, 1))(current_layer)
    act = Activation(activation_name)(final_convolution)
    model = Model(inputs=inputs, outputs=act)

    if not isinstance(metrics, list):
        metrics = [metrics]

    if include_label_wise_dice_coefficients and n_labels > 1:
        label_wise_dice_metrics = [get_label_dice_coefficient_function(index) for index in range(n_labels)]
        if metrics:
            metrics = metrics + label_wise_dice_metrics
        else:
            metrics = label_wise_dice_metrics

    model.compile(optimizer=Adam(lr=initial_learning_rate), loss=dice_coefficient_loss, metrics=metrics)
    return model


def create_convolution_block(input_layer, n_filters, batch_normalization=False, kernel=(3, 3, 3), activation=None,
                             padding='same', strides=(1, 1, 1), instance_normalization=False):
    """

    :param strides:
    :param input_layer:
    :param n_filters:
    :param batch_normalization:
    :param kernel:
    :param activation: Keras activation layer to use. (default is 'relu')
    :param padding:
    :return:
    """
    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides)(input_layer)
    if batch_normalization:
        layer = BatchNormalization(axis=1)(layer)
    elif instance_normalization:
        try:
            #J.Lee:original: from keras_contrib.layers.normalization import InstanceNormalization
            from tensorflow_addons.layers import InstanceNormalization
        except ImportError:
            raise ImportError("Install keras_contrib in order to use instance normalization."
                              "\nTry: pip install git+https://www.github.com/farizrahman4u/keras-contrib.git")
        layer = InstanceNormalization(axis=1)(layer)
    if activation is None:
        return Activation('relu')(layer)
    else:
        return activation()(layer)


def compute_level_output_shape(n_filters, depth, pool_size, image_shape):
    """
    Each level has a particular output shape based on the number of filters used in that level and the depth or number 
    of max pooling operations that have been done on the data at that point.
    :param image_shape: shape of the 3d image.
    :param pool_size: the pool_size parameter used in the max pooling operation.
    :param n_filters: Number of filters used by the last node in a given level.
    :param depth: The number of levels down in the U-shaped model a given node is.
    :return: 5D vector of the shape of the output node 
    """
    output_image_shape = np.asarray(np.divide(image_shape, np.power(pool_size, depth)), dtype=np.int32).tolist()
    return tuple([None, n_filters] + output_image_shape)


def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2), strides=(2, 2, 2),
                       deconvolution=False):
    if deconvolution:
        return Conv3DTranspose(filters=n_filters, kernel_size=kernel_size,
                               strides=strides) # J.Lee:original:Deconvolution3D
    else:
        return UpSampling3D(size=pool_size)


In [26]:
#brats/train.py

import os
import glob
import tensorflow
# from unet3d.data import write_data_to_file, open_data_file
# from unet3d.generator import get_training_and_validation_generators
# from unet3d.model import unet_model_3d
# from unet3d.training import load_old_model, train_model


config = dict()
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
config["image_shape"] = (144, 144, 144)  # This determines what shape the images will be cropped/resampled to.
config["patch_shape"] = (64, 64, 64)  # switch to None to train on the whole image
config["labels"] = (1, 2, 4)  # the label numbers on the input image
config["n_labels"] = len(config["labels"])
config["all_modalities"] = ["flair", "t1", "t1ce", "t2"]
config["training_modalities"] = config["all_modalities"]  # change this if you want to only use some of the modalities
config["nb_channels"] = len(config["training_modalities"])
if "patch_shape" in config and config["patch_shape"] is not None:
    config["input_shape"] = tuple([config["nb_channels"]] + list(config["patch_shape"]))
else:
    config["input_shape"] = tuple([config["nb_channels"]] + list(config["image_shape"]))
config["truth_channel"] = config["nb_channels"]
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["batch_size"] = 6
config["validation_batch_size"] = 12
config["n_epochs"] = 500  # cutoff the training after this many epochs
config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 50  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 0.00001
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["validation_split"] = 0.8  # portion of the data that will be used for training
config["flip"] = False  # augments the data by randomly flipping an axis during
config["permute"] = True  # data shape must be a cube. Augments the data by permuting in various directions
config["distort"] = None  # switch to None if you want no distortion
config["augment"] = config["flip"] or config["distort"]
config["validation_patch_overlap"] = 0  # if > 0, during training, validation patches will be overlapping
config["training_patch_start_offset"] = (16, 16, 16)  # randomly offset the first patch index by up to this offset
config["skip_blank"] = True  # if True, then patches without any target will be skipped

data_dir = 'X:/IAMEDIC/Data/MICCAI_BraTS_2019_Data_Training/'
config["data_file"] = data_dir + "h5_outfiles.h5" #os.path.abspath("brats_data.h5")
config["model_file"] = data_dir + "tumor_segmentation_model.h5" #os.path.abspath("tumor_segmentation_model.h5")
config["training_file"] = data_dir + "training_ids.pkl" #os.path.abspath("training_ids.pkl")
config["validation_file"] = data_dir + "validation_ids.pkl" #os.path.abspath("validation_ids.pkl")
config["overwrite"] = False  # If True, will previous files. If False, will use previously written files.


def fetch_training_data_files():
    training_data_files = list()
    for subject_dir in glob.glob(os.path.join(os.path.dirname(__file__), "data", "preprocessed", "*", "*")):
        subject_files = list()
        for modality in config["training_modalities"] + ["truth"]:
            subject_files.append(os.path.join(subject_dir, modality + ".nii.gz"))
        training_data_files.append(tuple(subject_files))
    return training_data_files


# def main(overwrite=False):
# convert input images into an hdf5 file
overwrite = config["overwrite"] 

if overwrite or not os.path.exists(config["data_file"]):
    training_files = fetch_training_data_files()

    write_data_to_file(training_files, config["data_file"], image_shape=config["image_shape"])
data_file_opened = open_data_file(config["data_file"])

if not overwrite and os.path.exists(config["model_file"]):
    model = load_old_model(config["model_file"])
else:
    # instantiate new model
    model = unet_model_3d(input_shape=config["input_shape"],
                          pool_size=config["pool_size"],
                          n_labels=config["n_labels"],
                          initial_learning_rate=config["initial_learning_rate"],
                          deconvolution=config["deconvolution"])

In [27]:
# get training and testing generators
train_generator, validation_generator, n_train_steps, n_validation_steps = get_training_and_validation_generators(
    data_file_opened,
    batch_size=config["batch_size"],
    data_split=config["validation_split"],
    overwrite=overwrite,
    validation_keys_file=config["validation_file"],
    training_keys_file=config["training_file"],
    n_labels=config["n_labels"],
    labels=config["labels"],
    patch_shape=config["patch_shape"],
    validation_batch_size=config["validation_batch_size"],
    validation_patch_overlap=config["validation_patch_overlap"],
    training_patch_start_offset=config["training_patch_start_offset"],
    permute=config["permute"],
    augment=config["augment"],
    skip_blank=config["skip_blank"],
    augment_flip=config["flip"],
    augment_distortion_factor=config["distort"])



Loading previous validation split...
Number of training steps:  346
Number of validation steps:  47


In [ ]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)

# run training
train_model(model=model,
            model_file=config["model_file"],
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=n_train_steps,
            validation_steps=n_validation_steps,
            initial_learning_rate=config["initial_learning_rate"],
            learning_rate_drop=config["learning_rate_drop"],
            learning_rate_patience=config["patience"],
            early_stopping_patience=config["early_stop"],
            n_epochs=config["n_epochs"])
data_file_opened.close()


# if __name__ == "__main__":
#     main(overwrite=config["overwrite"])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 346 steps, validate for 47 steps
Epoch 1/500
345/346 [============================>.] - ETA: 6s - loss: -0.1398 - dice_coefficient: 0.1398 